### Set up working env

In [131]:
import rafm, pandas as pd, numpy as np,matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec as gs
import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib import colormaps
from scipy.stats import mannwhitneyu as mwu, spearmanr,pearsonr
import glob,os
import peptides
import warnings

# define two reader funcitons
def read_fasta(fasta_file):
    content = {}
    fas = open(fasta_file,'r')
    seq = ''
    last_entry = ''
    while True:
        line = fas.readline()
        if len(line)==0:
            content[last_entry]=seq
            break
        if '>' in line:
            if last_entry=='':
                line = line.rstrip()
                last_entry=line
            else:
                line = line.rstrip()
                content[last_entry]=seq
                last_entry=line
                seq=''
        else:
            line = line.rstrip()
            seq += line
    return content

def flDPnn_reader(filename):
    f=open(filename,'r')
    flDPnn_dict ={}
    header = ''
    while True:
        line = f.readline().rstrip()
        if len(line)==0:
            break
        if line.startswith('>'):
            header = line[1:11]
            entry = []
            if header not in flDPnn_dict:
                for i in range(11):
                    entry.append(f.readline().rstrip().split(','))
                flDPnn_dict[header] = entry 
    return flDPnn_dict

### Load selected ortholog table

In [132]:
# Load data
ortho_table = pd.read_csv('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Final_draft/Tables/union_ortho_table_filtered.csv').drop(columns=['Unnamed: 0']).set_index('gene')
ortho_table['Locus']=ortho_table.index

### Download AlphaFold data for selected Msm homologs

In [133]:
# retrieve alphafold data
import os,glob
msm_proteome = pd.read_excel('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Important_references/Msm_proteome_20230816.xlsx')
msm_proteome['Locus'] = msm_proteome['Gene Names'].str.extract(r'(MSMEG_\d+)')#Create new column with just locus
msm_proteome.set_index('Locus',inplace=True)
msm_proteome['Locus'] = msm_proteome.index
msm_proteome_ingroup = msm_proteome.loc[ortho_table['Locus'].values].copy()

# downloading AF data
for x in msm_proteome_ingroup['Entry'].values:
    pdb_file = 'https://alphafold.ebi.ac.uk/files/AF-{}-F1-model_v4.pdb'.format(x)
    pae_file = 'https://alphafold.ebi.ac.uk/files/AF-{}-F1-predicted_aligned_error_v4.json'.format(x)
    cif_file = 'https://alphafold.ebi.ac.uk/files/AF-{}-F1-model_v4.cif'.format(x)
    pdb_export = '/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Alphafold/{}.pdb'.format(x)
    pae_export = '/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Alphafold/{}_pae.json'.format(x)
    cif_export = '/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Alphafold/{}.cif'.format(x)
    if (not os.path.isfile(pdb_export)) or (not os.path.isfile(pae_export)) or (not os.path.isfile(cif_export)):
        os.system(f'curl {pdb_file} -o {pdb_export}')
        os.system(f'curl {pae_file} -o {pae_export}')
        os.system(f'curl {cif_file} -o {cif_export}')

### flDPnn predictions were retrieved from webserver http://biomine.cs.vcu.edu/servers/flDPnn/

### Load protein features

In [134]:
warnings.filterwarnings('ignore')

plddt = []
prot_seq = []
# Load features generated by Ernst, ignore sasa as it is not normalized by protein length
msm_features = pd.read_csv('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/202308_BacPROTAC_prediction_model/Msm_proteome_feature_analysis.csv').drop(columns=['sasa'])
ortho_table['uniprot_id'] = msm_proteome.loc[ortho_table.index]['Entry']
#Add a locus column to the features df based on mapping uniprot ID
entry_to_locus = dict(zip(msm_proteome['Entry'], msm_proteome['Locus']))
msm_features['locus'] = msm_features['uniprot_id'].copy().map(entry_to_locus)


# load plDDT raw data
for i,(uid,l) in enumerate(ortho_table[['uniprot_id','Locus']].values):
    m = MMCIF2Dict('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Alphafold/{}.cif'.format(uid))
    plddt.append(np.array(m['_ma_qa_metric_local.metric_value']).astype(float))
ortho_table['pLDDT_raw'] = plddt

#load fLDPnn data
flDPnn_dict = {}
for f in sorted(glob.glob('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Important_references/flDPnn/*.csv')):
    flDPnn_dict.update(flDPnn_reader(f))

# flDPnn disordered domain prediction
ortho_table['flDPnn_disorder_propensity'] = [np.array(flDPnn_dict[x][2]).astype(float) for x in ortho_table['Locus'].values]


# load AF features generated by Ernst
g_features = []
for x in ortho_table['Locus'].values:
    if x in msm_features['locus'].values and 'MSMEG' in x: 
        subset = msm_features[msm_features['locus']==x]
        g_features.append(subset[msm_features.columns[1:-1]].values[0])
ortho_table[msm_features.columns[1:-1]] = np.array(g_features)

#calculate full length protein peptide feature stats
full_feature = pd.DataFrame([peptides.Peptide(s).descriptors() for s in ortho_table['Msm_seq'].values])
ortho_table[['Full_{}'.format(x) for x in full_feature.columns]] = full_feature.values


#calculate full length protein stats
for f in ['flDPnn_disorder_propensity','pLDDT_raw']:
    #ortho_table['{}_max'.format(f.split('_')[0])] = [x.min() for x in ortho_table[f].values]
    ortho_table['{}_mean'.format(f.split('_')[0])] = [x.mean() for x in ortho_table[f].values]
    #ortho_table['{}_median'.format(f.split('_')[0])] = [np.median(x) for x in ortho_table[f].values]
    #ortho_table['{}_Q1'.format(f.split('_')[0])] = [np.percentile(x,25) for x in ortho_table[f].values]
    #ortho_table['{}_Q3'.format(f.split('_')[0])] = [np.percentile(x,75) for x in ortho_table[f].values]
    #ortho_table['{}_IQR'.format(f.split('_')[0])] = [np.percentile(x,75)-np.percentile(x,25) for x in ortho_table[f].values]
    ortho_table['{}_STD'.format(f.split('_')[0])] = [np.std(x) for x in ortho_table[f].values]
    if f=='flDPnn_disorder_propensity':
        ortho_table['{}>0.2_count'.format(f.split('_')[0])] = [(x>0.2).sum() for x in ortho_table[f].values]
        ortho_table['{}>0.2_frac'.format(f.split('_')[0])] = [(x>0.2).sum()/len(x) for x in ortho_table[f].values]
        ortho_table['{}>0.5_frac'.format(f.split('_')[0])] = [(x>0.5).sum()/len(x) for x in ortho_table[f].values]
        ortho_table['{}>0.5_count'.format(f.split('_')[0])] = [(x>0.5).sum() for x in ortho_table[f].values]
    

for term_length in [15,30]:
    # terminal sequence peptide features
    nterm = [x[:term_length] for x in ortho_table['Msm_seq'].values]
    cterm = [x[-term_length:] for x in ortho_table['Msm_seq'].values]
    nterm_feature = pd.DataFrame([peptides.Peptide(s).descriptors() for s in nterm])
    cterm_feature = pd.DataFrame([peptides.Peptide(s).descriptors() for s in cterm])
    ortho_table[['Nterm{}_{}'.format(term_length,x) for x in nterm_feature.columns]] = nterm_feature.values
    ortho_table[['Cterm{}_{}'.format(term_length,x) for x in cterm_feature.columns]] = cterm_feature.values

    # calculate terminal disorderedness and pLDDT stats
    for f in ['flDPnn_disorder_propensity','pLDDT_raw']:
        #ortho_table['Nterm{}_{}_min'.format(term_length,f.split('_')[0])] = [x[:term_length].min() for x in ortho_table[f].values]
        ortho_table['Nterm{}_{}_mean'.format(term_length,f.split('_')[0])] = [x[:term_length].mean() for x in ortho_table[f].values]
        #ortho_table['Nterm{}_{}_median'.format(term_length,f.split('_')[0])] = [np.median(x[:term_length]) for x in ortho_table[f].values]
        #ortho_table['Nterm{}_{}_Q1'.format(term_length,f.split('_')[0])] = [np.percentile(x[:term_length],25) for x in ortho_table[f].values]
        #ortho_table['Nterm{}_{}_Q3'.format(term_length,f.split('_')[0])] = [np.percentile(x[:term_length],75) for x in ortho_table[f].values]
        #ortho_table['Nterm{}_{}_IQR'.format(term_length,f.split('_')[0])] = [np.percentile(x[:term_length],75)-np.percentile(x[:term_length],25) for x in ortho_table[f].values]
        ortho_table['Nterm{}_{}_std'.format(term_length,f.split('_')[0])] = [np.std(x[:term_length]) for x in ortho_table[f].values]
        #ortho_table['Cterm{}_{}_min'.format(term_length,f.split('_')[0])] = [x[-term_length:].min() for x in ortho_table[f].values]
        ortho_table['Cterm{}_{}_mean'.format(term_length,f.split('_')[0])] = [x[-term_length:].mean() for x in ortho_table[f].values]
        #ortho_table['Cterm{}_{}_median'.format(term_length,f.split('_')[0])] = [np.median(x[-term_length:]) for x in ortho_table[f].values]
        #ortho_table['Cterm{}_{}_Q1'.format(term_length,f.split('_')[0])] = [np.percentile(x[-term_length:],25) for x in ortho_table[f].values]
        #ortho_table['Cterm{}_{}_Q3'.format(term_length,f.split('_')[0])] = [np.percentile(x[-term_length:],75) for x in ortho_table[f].values]
        #ortho_table['Cterm{}_{}_IQR'.format(term_length,f.split('_')[0])] = [np.percentile(x[-term_length:],75)-np.percentile(x[-term_length:],25) for x in ortho_table[f].values]
        ortho_table['Cterm{}_{}_std'.format(term_length,f.split('_')[0])] = [np.std(x[-term_length:]) for x in ortho_table[f].values]
        if f=='flDPnn_disorder_propensity':
            ortho_table['Nterm{}_{}>0.2_frac'.format(term_length,f.split('_')[0])] = [(x[:term_length]>0.2).sum()/term_length for x in ortho_table[f].values]
            ortho_table['Cterm{}_{}>0.2_frac'.format(term_length,f.split('_')[0])] = [(x[-term_length:]>0.2).sum()/term_length for x in ortho_table[f].values]
        if f=='pLDDT_raw':
            ortho_table['Nterm{}_{}<50_frac'.format(term_length,f.split('_')[0])] = [(x[:term_length]<50).sum()/term_length for x in ortho_table[f].values]
            ortho_table['Cterm{}_{}<50_frac'.format(term_length,f.split('_')[0])] = [(x[-term_length:]<50).sum()/term_length for x in ortho_table[f].values]

### Save protein features

In [135]:
ortho_table.to_csv('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Final_draft/Tables/union_ortho_table_withFeatures.csv')

In [137]:
import pickle as pk
pk.dump(ortho_table,open('/Users/jzrolling/Desktop/Projects/HSPH/BacPROTAC/Final_draft/Tables/union_ortho_table_withFeatures.pk','wb'))